In [8]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from pandas import Series
from pandas import DataFrame
from sklearn.preprocessing import Imputer 
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
df = pd.read_csv("train.csv") 
X_test = pd.read_csv("test.csv")


df['Title'] = df['Name'].map(lambda name:name.split(',')[1].split('.')[0].strip())
# a map of more aggregated titles
Title_Dictionary = {
                        "Capt":       "Officer",
                        "Col":        "Officer",
                        "Major":      "Officer",
                        "Jonkheer":   "Royalty",
                        "Don":        "Royalty",
                        "Sir" :       "Royalty",
                        "Dr":         "Officer",
                        "Rev":        "Officer",
                        "the Countess":"Royalty",
                        "Dona":       "Royalty",
                        "Mme":        "Mrs",
                        "Mlle":       "Miss",
                        "Ms":         "Mrs",
                        "Mr" :        "Mr",
                        "Mrs" :       "Mrs",
                        "Miss" :      "Miss",
                        "Master" :    "Master",
                        "Lady" :      "Royalty"

                        }
    
# we map each title
df['Title'] = df.Title.map(Title_Dictionary)

X_test = pd.read_csv("test.csv")
X_test['Title'] = X_test['Name'].map(lambda name:name.split(',')[1].split('.')[0].strip())   
# we map each title
X_test['Title'] = X_test.Title.map(Title_Dictionary)


# encoding in dummy variable
titles_dummies = pd.get_dummies(df['Title'],prefix='Title')
df = pd.concat([df,titles_dummies],axis=1)
    
# removing the title variable
df.drop('Title',axis=1,inplace=True)

# encoding in dummy variable
titles_dummies = pd.get_dummies(X_test['Title'],prefix='Title')
X_test = pd.concat([X_test,titles_dummies],axis=1)
    
# removing the title variable
X_test.drop('Title',axis=1,inplace=True)

df = df.drop(['Ticket', 'Cabin', 'Name', 'Fare','PassengerId'],axis=1)
df = df.dropna()
y = df["Survived"]
X = df.drop("Survived", axis=1)

X_test = X_test.drop(['Ticket', 'Cabin', 'Name', 'Fare','PassengerId'],axis=1)

# sex and embarrked are object so we convert them to categorical data and then to numerical data using one hot encoding 
# 1. INSTANTIATE
# encode labels with value between 0 and n_classes-1.
# le = preprocessing.LabelEncoder()
# X_test['Sex']=le.fit_transform(X_test.__getattr__('Sex'))
# X_test['Embarked']=le.fit_transform(X_test.__getattr__('Embarked'))

# X['Sex']=le.fit_transform(X.__getattr__('Sex'))
# X['Embarked']=le.fit_transform(X.__getattr__('Embarked'))
# X = X.apply(le.fit_transform)
X["male"]= np.where(X['Sex']=="male", 1, 0)
X["female"]= np.where(X['Sex']=="female", 1, 0)
X["S"]= np.where(X['Embarked']=="S", 1, 0)
X["C"]= np.where(X['Embarked']=="C", 1, 0)
X["Q"]= np.where(X['Embarked']=="Q", 1, 0)
X["First"]= np.where(X['Pclass']==1, 1, 0)
X["Second"]= np.where(X['Pclass']==2, 1, 0)
X["Third"]= np.where(X['Pclass']==3, 1, 0)

X = X.drop('Pclass',axis=1)

X_test["male"]= np.where(X_test['Sex']=="male", 1, 0)
X_test["female"]= np.where(X_test['Sex']=="female", 1, 0)
X_test["S"]= np.where(X_test['Embarked']=="S", 1, 0)
X_test["C"]= np.where(X_test['Embarked']=="C", 1, 0)
X_test["Q"]= np.where(X_test['Embarked']=="Q", 1, 0)
X_test["First"]= np.where(X_test['Pclass']==1, 1, 0)
X_test["Second"]= np.where(X_test['Pclass']==2, 1, 0)
X_test["Third"]= np.where(X_test['Pclass']==3, 1, 0)

X_test = X_test.drop('Pclass',axis=1)
X = X.drop("Embarked", axis=1)
X = X.drop("Sex", axis=1)
X_test = X_test.drop("Embarked", axis=1)
X_test = X_test.drop("Sex", axis=1)

scaler = MinMaxScaler() 
scaled_values = scaler.fit_transform(X) 
X.loc[:,:] = scaled_values
X

my_imputer = Imputer()
X_test = my_imputer.fit_transform(X_test)
X_test= scaler.transform(X_test)
X_test
model = RandomForestClassifier(n_estimators=100, max_features =7, max_depth=3, random_state=3)
# model.fit(X,y)
# model.score(X,y)
# result = model.predict(X_test)
# pd.Series(result).to_csv('svc_rbf_normalized_gamma_c.csv')
cv_scores = cross_val_score(model, X, y, cv=8)
print(cv_scores)
print(np.mean(cv_scores))

# tried with logistic regression till now without one hot encoding. Trying with one hot encoding now 

# TODO: create a OneHotEncoder object, and fit it to all of X
# 1. INSTANTIATE
# enc = preprocessing.OneHotEncoder()
# # # 2. FIT
# enc.fit(X)
# # 3. Transform
# X = enc.transform(X)
# model = LogisticRegression()
model.fit(X,y)
score = model.score(X, y)
print(score)
result = model.predict(X_test)
pd.Series(result).to_csv('randomforest2.csv')

[ 0.80898876  0.82022472  0.83146067  0.80898876  0.79775281  0.76404494
  0.82022472  0.87640449]
0.816011235955
0.824438202247
